In [1]:
from docx import Document
import docx
import os
from docx.document import Document as doctwo
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import pandas as pd

In [2]:
document = Document('PandasTableExtraction.docx')

In [3]:
rels = {}
for r in document.part.rels.values():
    if isinstance(r._target, docx.parts.image.ImagePart):
        rels[r.rId] = os.path.basename(r._target.partname)
rels

{'rId7': 'image2.jpg', 'rId6': 'image1.jpg'}

In [4]:
##This function extracts the tables and paragraphs from the document object
def iter_block_items(parent):
    """
    Yield each paragraph and table child within *parent*, in document order.
    Each returned value is an instance of either Table or Paragraph. *parent*
    would most commonly be a reference to a main Document object, but
    also works for a _Cell object, which itself can contain paragraphs and tables.
    """
    if isinstance(parent, doctwo):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

In [5]:
for block in iter_block_items(document):
    if 'text' in str(block):
        if block.style.name.startswith('Heading'):
            print('[Heading]', block.text)
        elif 'Graphic' in block._p.xml:
        # Get the rId of the image
            for rId in rels:
                if rId in block._p.xml:
                    print('[Image]', os.path.join('img', rels[rId]))
        else:
            print('-', block.text)
    else:
        print('[Table]')

[Heading] Test Document
- This document was created just to test the simple table extraction into pandas dataframes.
[Heading] Tables
[Heading] Simple Table with heading
- Simple tables have a uniform number of columns and rows, without any merged cells. This table contains heading
[Table]
- 
[Heading] Simple Table with no heading
- Simple tables have a uniform number of columns and rows, without any merged cells. This table contains no heading
[Table]
- 
[Heading] Complex Tables
- The following is a complex table, using merged cells as headers 
[Table]
-  
- 
- 
[Heading] Image 1
- Following is an image
[Image] img\image1.jpg
[Heading] Image 2
- Following is another image  
[Image] img\image2.jpg
- 
